# Backend Validation: Module 1 - DataIngest

This notebook validates the `DataIngest` module of the FinPattern-Engine.

**Objective:**
- Ingest the provided EUR/USD tick data.
- Generate 100-tick, 1000-tick, and 1-minute bars.
- Verify the output files and the manifest.

In [ ]:
import sys
import os
import json
import pandas as pd

# Add the project root to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from core.data_ingest.data_ingest import run as data_ingest_run

# Configuration for the DataIngest module
config = {
    "out_dir": "/home/ubuntu/FinPattern-Engine/output/data_ingest",
    "csv": {
        "path": "/home/ubuntu/FinPattern-Engine/data/EURUSD-2025-07.csv"
    },
    "symbol": "EURUSD",
    "price_basis": "mid",
    "max_missing_gap_seconds": 60,
    "trim_weekend": True,
    "bar_frames": [
        {"type": "time", "unit": "1m"},
        {"type": "tick", "count": 100},
        {"type": "tick", "count": 1000}
    ],
    "pip_size": 0.0001
}

# Run the DataIngest module
data_ingest_output = data_ingest_run(config)

# Print the output manifest
print(json.dumps(data_ingest_output, indent=2))

In [ ]:
# Verify the output files
output_dir = config["out_dir"]
!ls -l {output_dir}

# Load and inspect one of the generated bar files
bars_1m_path = data_ingest_output["frames"]["1m"]["path"]
df_bars_1m = pd.read_parquet(bars_1m_path)
print("1-minute bars:")
print(df_bars_1m.head())
print(df_bars_1m.info())

# Load and inspect the quality report
quality_report_path = data_ingest_output["quality_report"]
with open(quality_report_path, 'r') as f:
    quality_report = json.load(f)
print("\nQuality Report:")
print(json.dumps(quality_report, indent=2))

# Load and inspect the manifest
manifest_path = data_ingest_output["manifest"]
with open(manifest_path, 'r') as f:
    manifest = json.load(f)
print("\nManifest:")
print(json.dumps(manifest, indent=2))